

---


**Notebook pour le training de modèle pour le PTRANS**

☕*Created by :*

*   Mattéo Boursault
*   Ibrahim Braham
*   Adam Creusevault

Dernière modification : *06/05/2022*

---



**pour laisser tourner sur une longue durée**

```
var startClickConnect = function startClickConnect(){
    var clickConnect = function clickConnect(){
        console.log("Connnect Clicked - Start");
        document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
        console.log("Connnect Clicked - End"); 
    };

    var intervalId = setInterval(clickConnect, 180000);

    var stopClickConnectHandler = function stopClickConnect() {
        console.log("Connnect Clicked Stopped - Start");
        clearInterval(intervalId);
        console.log("Connnect Clicked Stopped - End");
    };
```

In [ ]:
'''
- Complétez le chemin des 'Verite_terrain'
- Modifiez les paramètres dans la section Configuration au besoin
- Executez toutes les cellules du Notebook
- Have fun
'''
VERITE_PATH = 'drive/MyDrive/PTRANS/ptrans-main/Dev/data/'

#### Import + Connexion au Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install albumentations==0.4.6
!pip install torchinfo
!CUDA_LAUNCH_BLOCKING=1 # pour debug CUDA

     |████████████████████████████████| 117 kB 7.8 MB/s 
     |████████████████████████████████| 948 kB 52.0 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=4adb53f10ff374698ef3d9ae7376d535a63a3cce89dc84044a734061c972f914
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from pynvml import *
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
import time
import copy
import torch.nn as nn

In [ ]:
# pour afficher la mémoire libre/utilisée de la GPU 
# pour lancer le training avec une taille de radios importante, il faut vérifier que toute la mémoire de la GPU est libre sinon redémarrer l'environnement d'exécution.
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}') 
print(f'free     : {info.free}')
print(f'used     : {info.used}') 

total    : 17071734784
free     : 17071734784
used     : 0


#### Dataset

In [ ]:
class RadioDataset(Dataset):
    def __init__(self, inputs_chiens, inputs_chats, transform=None):
        super().__init__()

        self.transform = transform

        # Chemins des dossiers de la vérité terrain
        ROOT = VERITE_PATH + 'Verite_terrain/'
        DOG_PATH = 'Chiens/'
        CAT_PATH = 'Chats/'
        RADIOS = 'Radios/'
        HEART_MASKS = 'Coeur/'
        VTB_MASKS = 'Vertebres/'
        PROCESS_MASKS = 'Process_epineux/'

        # Récupération et stockage des chemins des fichiers
        dog_radios = self.generate_chemin(ROOT + DOG_PATH + RADIOS, inputs_chiens)
        cat_radios = self.generate_chemin(ROOT + CAT_PATH + RADIOS, inputs_chats)

        dog_hearts = self.generate_chemin(ROOT + DOG_PATH + HEART_MASKS, inputs_chiens, True)
        cat_hearts = self.generate_chemin(ROOT + CAT_PATH + HEART_MASKS, inputs_chats, True)

        dog_vtb = self.generate_chemin(ROOT + DOG_PATH + VTB_MASKS, inputs_chiens, True)
        cat_vtb = self.generate_chemin(ROOT + CAT_PATH + VTB_MASKS, inputs_chats, True)

        dog_prc = self.generate_chemin(ROOT + DOG_PATH + PROCESS_MASKS, inputs_chiens, True)
        cat_prc = self.generate_chemin(ROOT + CAT_PATH + PROCESS_MASKS, inputs_chats, True)

        self.radios_arr = dog_radios + cat_radios
        self.heart_arr = dog_hearts + cat_hearts
        self.vtb_arr = dog_vtb + cat_vtb
        self.process_arr = dog_prc + cat_prc

        self.data_len = len(self.radios_arr)

        print("GROUND_TRUTH FOUND : ", self.data_len)

    def __getitem__(self, idx):
        """
        Fonction obligatoire utilisée par le loader lors de l'itération du dataset
        Récupération du chemin et chargement de la vérité terrain puis labellisation
        """

        radio = np.array(Image.open(self.radios_arr[idx]))

        heart_mask = self.grayscale(self.heart_arr[idx])
        vtb_mask = self.grayscale(self.vtb_arr[idx])
        process_mask = self.grayscale(self.process_arr[idx])

        heart_mask[heart_mask != 0] = 1
        vtb_mask[vtb_mask != 0] = 2
        process_mask[process_mask != 0] = 3
        #masks = heart_mask + vtb_mask + process_mask

        
        masks = np.zeros_like(heart_mask)
        masks[heart_mask == 1] = 1
        masks[vtb_mask == 2] = 2
        masks[process_mask == 3] = 3
        

        if self.transform is not None:
            augmentations = self.transform(image=radio,mask=masks)
            radio = augmentations['image']
            masks = augmentations['mask']

        grayscale_transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                                  transforms.ToTensor()])
        radio = Image.fromarray(radio)
        radio = grayscale_transform(radio)

        """
        background = np.zeros_like(masks) 
        heart_mask = np.zeros_like(masks)
        vtb_mask = np.zeros_like(masks)
        process_mask = np.zeros_like(masks)

        background[masks == 0] = 1
        heart_mask[masks == 1] = 1
        vtb_mask[masks == 2] = 1
        process_mask[masks == 3] = 1
        
        #masks = torch.tensor([heart_mask, vtb_mask, process_mask, background])
        masks = np.array([heart_mask, vtb_mask, process_mask, background])
        """

        """
        Utiliser from_numpy plutôt que toTensor permet de ne pas normaliser les
        données afin de respecter la labellisation des classes par 0, 1, 2 et 3
        """
        return {"image": radio, "mask": torch.from_numpy(masks).to(DEVICE)}


    def __len__(self):
        """
        Méthode nécessaire au loader qui retourne le
        nombre de données dans le dataset
        """
        return self.data_len


    def grayscale(self, path):
        image = Image.open(path)
        image.load()
        background = Image.new("RGB", image.size, (0, 0, 0))
        background.paste(image, mask=image.split()[3])
        image = background
        image = image.convert('L')
        return np.array(image)

    def generate_chemin(self, path, tableau, jpgTopng=False):
        tab = []
        if (jpgTopng):
          for i in range(len(tableau)):
            tab.append(path + tableau[i][0:-3] + 'png')
        else:
          for i in range(len(tableau)):
            tab.append(path + tableau[i])
        return tab

#### Modèle

In [ ]:
# modele UNet :

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Transposed convolution then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels , out_channels, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)


    def forward_padding(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = nn.functionnal.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

    def forward(self, target_tensor, contracting_tensor):
        target_tensor = self.up(target_tensor)
        target_height = target_tensor.size()[2]
        target_width = target_tensor.size()[3]
        crop = transforms.CenterCrop((target_height, target_width))

        contracting_tensor = crop(contracting_tensor)
        new_tensor = torch.cat((target_tensor, contracting_tensor), 1)

        return self.conv(new_tensor)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
# nouveau modèle  avec 124 millions de paramètres
import torch
import torch.nn as nn


def double_conv(in_c, out_c):
    conv = nn.Sequential(
        nn.Conv2d(in_c, out_c, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True)
    )
    return conv.to(DEVICE)

# def addPadding(srcShapeTensor, tensor_whose_shape_isTobechanged):

#     if(srcShapeTensor.shape != tensor_whose_shape_isTobechanged.shape):
#         target = torch.zeros(srcShapeTensor.shape)
#         target[:, :, :tensor_whose_shape_isTobechanged.shape[2],
#                :tensor_whose_shape_isTobechanged.shape[3]] = tensor_whose_shape_isTobechanged
#         return target.to(DEVICE)
#     return tensor_whose_shape_isTobechanged.to(DEVICE)

class UNet2(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet2, self).__init__()
        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_1 = double_conv(3, 64)
        self.down_conv_2 = double_conv(64, 128)
        self.down_conv_3 = double_conv(128, 256)
        self.down_conv_4 = double_conv(256, 512)
        self.down_conv_5 = double_conv(512, 1024)

# new by Braham
        self.down_conv_6 = double_conv(1024, 2048)
        
        self.up_trans_0 = nn.ConvTranspose2d(
            in_channels=2048,
            out_channels=1024,
            kernel_size=2,
            stride=2
        )
        self.up_conv_0 = double_conv(2048, 1024)
# fin de new

        self.up_trans_1 = nn.ConvTranspose2d(
            in_channels=1024,
            out_channels=512,
            kernel_size=2,
            stride=2
        )
        self.up_conv_1 = double_conv(1024, 512)

        self.up_trans_2 = nn.ConvTranspose2d(
            in_channels=512,
            out_channels=256,
            kernel_size=2,
            stride=2
        )
        self.up_conv_2 = double_conv(512, 256)

        self.up_trans_3 = nn.ConvTranspose2d(
            in_channels=256,
            out_channels=128,
            kernel_size=2,
            stride=2
        )
        self.up_conv_3 = double_conv(256, 128)

        self.up_trans_4 = nn.ConvTranspose2d(
            in_channels=128,
            out_channels=64,
            kernel_size=2,
            stride=2
        )
        self.up_conv_4 = double_conv(128, 64)

        self.out = nn.Conv2d(
            in_channels=64,
            out_channels=n_classes,
            kernel_size=1
        )

    def forward(self, image):
        x1 = self.down_conv_1(image)
        x2 = self.max_pool_2x2(x1)
        x3 = self.down_conv_2(x2)
        x4 = self.max_pool_2x2(x3)
        x5 = self.down_conv_3(x4)
        x6 = self.max_pool_2x2(x5)
        x7 = self.down_conv_4(x6)        
        x8 = self.max_pool_2x2(x7)
        x9 = self.down_conv_5(x8)

# new by Braham   
        x10 = self.max_pool_2x2(x9)
        x11 = self.down_conv_6(x10)  
        x = self.up_trans_0(x11)
        x = self.up_conv_0(torch.cat([x9, x], 1))        
# fin de new
        
        x = self.up_trans_1(x9)
        x = self.up_conv_1(torch.cat([x7, x], 1))        
        x = self.up_trans_2(x)
        x = self.up_conv_2(torch.cat([x5, x], 1))   
        x = self.up_trans_3(x)
        x = self.up_conv_3(torch.cat([x3, x], 1))
        x = self.up_trans_4(x)
        x = self.up_conv_4(torch.cat([x1, x], 1))
        x = self.out(x)
        return x.to(DEVICE)

#### Configuration

In [ ]:
MODELE = UNet                 # modèle utilisé, voir dans la partie Modèle du notebook la liste des modèles disponibles
TRAINED_MODEL = None          # definit le nom du modèle que vous souhaitez réentrainer, si ce n'est pas cas None
PATH_SAVE = 'drive/MyDrive/'  # definit le chemin du dossier qui contient le modèle que vous souhaitez réentrainer
NB_CLASSES = 4                # 4 pour : coeur, vertebres, process_epineux, fond
NB_CHANNELS = 1               # un vecteur en sortie (la prédiction du modèle)
LEARNING_RATE = 0.001         # le taux d'apprentissage
TRAIN_SIZE = 0.8              # le % de segmentations allouées au jeu d'entrainement (1-TRAIN_SIZE pour la taille du jeu de validation)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # pour utiliser CUDA si nécessaire, ne pas toucher
BATCH_SIZE = 1                # la taille de votre batch
EPOCHS = 1000                 # le nombre de fois ou vous souhaitez itérer sur l'ensemble du jeu de donnée pour entrainer votre modèle
SIZE = 1472                   # la taille souhaité des radio en entrée du modèle (max 572 sans colab pro, 1520 avec), doit être un multiple de 16 (prérequis du modèle UNet)

#### Data Augmentations (with Albumentation)

In [ ]:
# les transformations à appliquer à partir de la bibliothèque Albumentations qui vont servir à resize les radios et les masques en dimension unique
# la réduction des dimensions va permettre que le training se déroule sans plantage de la GPU dû à un débordement de mémoire  
# et aussi permettre la data augmentation en appliquant un nombre important d'epochs car les transformations sont aléatoires.

train_transform = A.Compose([
    A.Rotate(limit=10,p=0.3),
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.Transpose(p=0.3),
    #A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, brightness_by_max=True, always_apply=False, p=0.3),
    A.GridDistortion(p=0.2),
    A.LongestMaxSize(SIZE),
    A.PadIfNeeded(min_height=SIZE, min_width=SIZE, border_mode=cv2.BORDER_CONSTANT),
], additional_targets={'mask': 'image'})

validation_transform = A.Compose([
    A.LongestMaxSize(SIZE),
    A.PadIfNeeded(min_height=SIZE, min_width=SIZE, border_mode=cv2.BORDER_CONSTANT),
], additional_targets={'mask': 'image'})

#### Création des Datasets

In [ ]:
# on a éclaté les data (radios et masques) en data train et data validation
from sklearn.model_selection import train_test_split

inputs_chiens = os.listdir('drive/MyDrive/PTRANS/ptrans-main/Dev/data/Verite_terrain/Chiens/Radios')
inputs_chats = os.listdir('drive/MyDrive/PTRANS/ptrans-main/Dev/data/Verite_terrain/Chats/Radios')

chiens_train, chiens_test = train_test_split(inputs_chiens, train_size=TRAIN_SIZE, random_state = 79)
chats_train, chats_test = train_test_split(inputs_chats, train_size=TRAIN_SIZE, random_state = 79)

# création des datasets pour le train et pour la validation
# création des dataloaders pour le train et pour la validation

train_data = RadioDataset(chiens_train, chats_train, train_transform)
valid_data = RadioDataset(chiens_test, chats_test, validation_transform)
train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
valid_dataloader = DataLoader(valid_data,batch_size=BATCH_SIZE,shuffle=False)

GROUND_TRUTH FOUND :  205
GROUND_TRUTH FOUND :  52


#### Lancement du training

In [ ]:
# la fonction fit déroule le training 
def fit(model,dataloader,data,optimizer,criterion):
    print('-------------Training---------------')
    model.train()
    train_running_loss = 0.0
    counter = 0
    
    # num of batches
    num_batches = int(len(data)/dataloader.batch_size)
    for i,data in tqdm(enumerate(dataloader),total = num_batches):
        counter+=1
        image,mask = data["image"].to(DEVICE), data["mask"].type('torch.LongTensor').to(DEVICE)
        optimizer.zero_grad()
        outputs = model(image)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs,mask)
        train_running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/counter
    return train_loss

# la fonction validate évalue le modèle à la fin de chaque epoch
def validate(model,dataloader,data,criterion):
    print("\n--------Validating---------\n")
    model.eval()
    valid_running_loss = 0.0
    counter = 0
    # number of batches
    num_batches = int(len(data)/dataloader.batch_size)
    with torch.no_grad():
        for i,data in tqdm(enumerate(dataloader),total=num_batches):
            counter+=1
            image,mask = data["image"].to(DEVICE), data["mask"].type('torch.LongTensor').to(DEVICE)
            outputs = model(image)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs,mask)
            valid_running_loss += loss.item()
    valid_loss = valid_running_loss/counter
    return valid_loss

In [ ]:
# Execution du training et sauvegarde du meilleur modèle et de la courbe des Loss
path_save = "drive/MyDrive/model.pth"

# Pour repartir d'un modèle déjà entrainé
if TRAINED_MODEL != None:
  model_path = PATH_SAVE + TRAINED_MODEL
  trained_model = torch.load(model_path)
  model = UNet(NB_CHANNELS, NB_CLASSES).to(DEVICE)
  model.load_state_dict(trained_model["model_state_dict"])

  train_array_loss = trained_model["train_array_loss"]
  val_array_loss = trained_model["val_array_loss"]
  lowest_val_loss = min(val_array_loss)

else:
  model = UNet(NB_CHANNELS, NB_CLASSES).to(DEVICE)

  lowest_val_loss = 10.
  train_array_loss = []
  val_array_loss = []

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss() # mais la BCEWithLogitsLoss est plus performante

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1} of {EPOCHS}")
    train_epoch_loss = fit(model, train_dataloader, train_data, optimizer, criterion)
    val_epoch_loss = validate(model, valid_dataloader, valid_data, criterion)
    train_array_loss.append(train_epoch_loss)
    val_array_loss.append(val_epoch_loss)
    print(f'Train Loss: {train_epoch_loss:.4f}')
    print(f'Val Loss: {val_epoch_loss:.4f}')
    print(f'Actual best val loss: {min(val_array_loss):.4f}')
    if val_epoch_loss < lowest_val_loss:  # On ne sauvegarde le modèle que si on réduit la validation Loss  
        lowest_val_loss= val_epoch_loss
        torch.save({
        'epoch': EPOCHS,
        'train_array_loss': train_array_loss,
        'val_array_loss': val_array_loss,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': criterion,
          }, path_save)

#### Test du modèle

In [ ]:
# define the model you want to test
MODEL_PATH = PATH_SAVE + 'model.pth'

In [ ]:
model = UNet(1, 4)
model_to_test = torch.load(MODEL_PATH)
model.load_state_dict(model_to_test["model_state_dict"])
model.eval()
model.to(DEVICE)

train_array_loss = model_to_test["train_array_loss"]
val_array_loss = model_to_test["val_array_loss"]

In [ ]:
def display_image_grid(model, dataset, nb_display_img = 10):

  cols = 3
  rows = nb_display_img
  figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(20, 100)) #10*nb_display_img

  for i in range(nb_display_img):

    radio = valid_data[i]["image"]
    mask = valid_data[i]["mask"].to('cpu')
    output = model(radio.unsqueeze(1).to(DEVICE))
    output = output[0].cpu().data.numpy()
    index = output.argmax(axis=0)
    index = index*100

    ax[i, 0].imshow(radio[0])
    ax[i, 1].imshow(mask)
    ax[i, 2].imshow(index)

    ax[i, 0].set_title("Image: ", fontsize=3*nb_display_img)
    ax[i, 1].set_title("Truth mask: ", fontsize=3*nb_display_img)
    ax[i, 2].set_title("predicted mask: ", fontsize=3*nb_display_img)

    ax[i, 0].set_axis_off()
    ax[i, 1].set_axis_off()
    ax[i, 2].set_axis_off()

  plt.tight_layout()
  plt.show()

  return

In [ ]:
def display_loss_curve(train_array_loss, val_array_loss):
  best_val_loss = min(val_array_loss)
  # loss plots
  plt.figure(figsize=(10, 7))
  plt.plot(train_array_loss, color="blue", label='train loss')
  plt.plot(val_array_loss, color="red", label='validation loss')
  plt.plot([val_array_loss.index(best_val_loss)], [best_val_loss], 'go', label="Best model")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

In [ ]:
summary(model, (1, 1, SIZE, SIZE))

In [ ]:
display_loss_curve(train_array_loss, val_array_loss)

In [ ]:
len(val_array_loss)

In [ ]:
display_image_grid(model, train_data, nb_display_img = 20)